In [ ]:
# !pip3 install holidays

In [1]:
import pandas as pd
import holidays
import datetime

In [3]:
df = pd.read_csv('/home/faye/data/vehicles_2018_cleaned_dataset')

In [4]:
# Converting date columns to datetime
datetime_columns = ['DAYOFSERVICE', 'LASTUPDATE']
for column in datetime_columns:
    df[column] = pd.to_datetime(df[column])

In [5]:
# Getting the Irish holidays 
# https://towardsdatascience.com/5-minute-guide-to-detecting-holidays-in-python-c270f8479387
irish_holidays_2018 = []
for date in holidays.Ireland(years=2018).items():
    irish_holidays_2018.append(str(date[0]))

In [6]:
irish_holidays_2018

['2018-01-01',
 '2018-12-25',
 '2018-03-17',
 '2018-03-19',
 '2018-04-02',
 '2018-05-07',
 '2018-06-04',
 '2018-08-06',
 '2018-10-29',
 '2018-12-26']

In [35]:
df['Is_Holiday'] = [1 if str(val).split()[0] in irish_holidays_2018 else 0 for val in df['DAYOFSERVICE']]
df['Is_Weekday'] = [1 if int(val.weekday()) < 5 else 0 for val in df['DAYOFSERVICE']]

In [39]:
df.tail(60)

,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,Is_Holiday,Is_Weekday
272562,2018-12-29,3265705,175605,0 days 11:39:10,2019-01-16 18:00:42,0,0
272563,2018-12-29,3265707,151209,0 days 07:20:37,2019-01-16 18:00:42,0,0
272564,2018-12-29,3265708,414128,0 days 19:14:35,2019-01-16 18:00:42,0,0
272565,2018-12-29,3265709,255788,0 days 12:58:36,2019-01-16 18:00:42,0,0
272566,2018-12-29,3265711,188117,0 days 12:02:11,2019-01-16 18:00:42,0,0
272567,2018-12-29,3265712,222613,0 days 14:04:17,2019-01-16 18:00:42,0,0
272568,2018-12-29,3265713,117685,0 days 08:15:11,2019-01-16 18:00:42,0,0
272569,2018-12-29,3265714,247389,0 days 17:28:36,2019-01-16 18:00:42,0,0
272570,2018-12-29,3265715,266555,0 days 14:51:06,2019-01-16 18:00:42,0,0
272571,2018-12-29,3265716,46096,0 days 03:37:57,2019-01-16 18:00:42,0,0
